In [1]:
ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']
import matplotlib.pyplot as plt
import numpy as np

test = np.array([
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
])


# # Normalize the values to the range 0-1 for better visualization
# plt.imshow(test, cmap='gray', interpolation='nearest')
# plt.colorbar()  # To show the scale
# plt.show()

In [5]:
test = None
if test == None:
    print("IS none")
elif test == []:
    print("Is empty")
else:
    print("Is full")

IS none
